In [1]:
try:
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    print('libraries are found')
except:
    print('Some libraries need to be installed')


libraries are found


In [13]:
# function to read the data
def read_data(path,file_name):
    df = pd.read_hdf(path + filename)
    return df
filename = 'data_train_v7.h5'
path = '/workspace/giacomo/bh_parts_forecasting/data/train/'
data = read_data(path,filename)

In [14]:
# function to treat the missing values
## remove columns which contain more than 50% of null values
def treat_null_values(df):
    
    df = df.dropna(thresh=df.shape[0]*0.5, how='all',axis=1) # -3
    df = df.dropna(axis=0,subset=['RECEIPT_DATE','BOOKED_DATE'])
    
    df = df[df['RECEIPT_DATE']>df['BOOKED_DATE']] # false
       
    return df

df1 = treat_null_values(data)

In [ ]:
# optional function
#  This funtion is to analyse the different types of columns # optional
def analyze_data_types(df):
    date_columns_df = df.select_dtypes(include=['datetime64'])
    object_columns_df = df.select_dtypes(include=['object'])
    numerics= [ 'int64','float64']
    numeric_columns_df = df.select_dtypes(include = numerics)
    
    return date_columns_df, object_columns_df,numeric_columns_df

date_df, objects_df,numeric_df=analyze_data_types(df1)

In [ ]:
# apply feature engg here after this step
# first apply on numeric_df columns
# second on objects_df columns
# third on date_df separately 
# for convenience 

## Feature Engineering - Numeric Columns 

In [ ]:
# we can remove UNIT_PRICE_USD as already AMOUNT_USD is there
# we can remove 'ORG_ID',ATTRIBUTE_NUMBER1', 'ATTRIBUTE_NUMBER2',ATTRIBUTE_NUMBER6', 'ATTRIBUTE_NUMBER7', 'ATTRIBUTE_NUMBER8' as they have 
# only one unique value

# see SO_LINE_FIRST and SO_LINE_LAST retain one col by finding the difference

In [15]:
def drop_features(df,cols_to_drop):
    df['diff_SO_LINE_FIRST_SO_LINE_LAST'] = df['SO_LINE_LAST'] - df['SO_LINE_FIRST']

    df.drop(cols_to_drop,axis=1,inplace=True)
    
    return df

columns_to_remove = ['ORG_ID','ATTRIBUTE_NUMBER1','ATTRIBUTE_NUMBER2','ATTRIBUTE_NUMBER6','ATTRIBUTE_NUMBER7','ATTRIBUTE_NUMBER8',
                    'UNIT_PRICE_USD','SO_LINE_FIRST','SO_LINE_LAST' ]
df1 = drop_features(df1,columns_to_remove)

## Feature Engineering - Objects Columns 

In [16]:

def treat_objects(df,col_to_drop):
    
    df.drop(col_to_drop,axis=1,inplace=True)
    
    object_columns_df = df.select_dtypes(include=['object'])
    objects_col = list(object_columns_df.columns)
    

    # label encode the categorical variables
    label_encoder = LabelEncoder()
    for col in objects_col:
         df[col] = label_encoder.fit_transform(df[col])
            
    return df
cols_to_remove = ['PO_STRING','PROMISED_DATE']
df1 = treat_objects(df1,cols_to_remove)

## Feature Engineering - Date Columns

In [18]:
# function to re-engineer the dates
def difference_dates(df,col1,col2):
            
    df['diff_'+col1+col2]=(df[col1]-df[col2]).dt.days
    df.drop([col1,col2],axis=1,inplace = True)
    
    return df
# call function on the required features    
df1 = difference_dates(df1,'RTS_','RTS_FINAL')
df1 = difference_dates(df1,'SO_CWD','SO_CREATION_DATE')
df1 = difference_dates(df1,'EOQ_CONTR','EOQ_REQUEST')
df1 = difference_dates(df1,'CONTRACTUAL_DATE','FULFILLMENT_CONTRACTUAL_DATE')
df1 = difference_dates(df1,'PO_FIRST_APPROVED_DATE','POR_APPROVED_DATE')
df1 = difference_dates(df1,'RECEIPT_DATE','BOOKED_DATE') 


# since most of the rows are same as NEED_BY_DATE','NEED_DATE_FIRST', 'NEED_DATE_LAST'
# 'NEED_BY_DATE' has same values as 'SUP_PO_NEED_BY_DATE'
# instead of these 3 features only one feature can be used and name it 'NEED_DATE_'
# 'DATE_WHEN_NEED_REV'
df1['NEED_DATE_'] = df1['NEED_BY_DATE']
df1.drop(['NEED_DATE_FIRST', 'NEED_DATE_LAST','SUP_PO_NEED_BY_DATE','DATE_WHEN_NEED_REV',
         'NEED_BY_DATE','PROMISED_DATE_LAST'],axis=1,inplace=True)



# both SCHEDULE_SHIP_DATE and 'LINE_REQUEST_DATE' are greater than NEED_DATE so take the diff
df1['diff_SCHEDULE_SHIP_DATE_NEED_DATE_'] = (df1['SCHEDULE_SHIP_DATE']-df1['NEED_DATE_']).dt.days
df1['diff_LINE_REQUEST_DATE_NEED_DATE_'] = (df1['LINE_REQUEST_DATE']-df1['NEED_DATE_']).dt.days
df1.drop(['LINE_REQUEST_DATE','SCHEDULE_SHIP_DATE'],axis=1,inplace=True)
#df1.drop('PROMISED_DATE',axis=1,inplace=True)  # this not datetime data type, and we drop this retaining PROMISED FIRST DATE

## Save the data 

In [27]:
def save_data(df,file_name_train,file_name_test):
    
    data_cleaned = df.dropna()
    data_cleaned =data_cleaned.reset_index(drop=True)
    train_data,test_data = train_test_split(data_cleaned,test_size = 0.2)
    train_data.to_csv(file_name_train,index=False)
    test_data.to_csv(file_name_test,index=False)
    
    return True
file1 = 'train_data_E2.csv'
file2 = 'test_data_E2.csv'
save_data(df1,file1,file2)
save_data(df1,file1,file2)